In [42]:
# https://www.lazard.com/research-insights/2023-levelized-cost-of-energyplus/

In [2]:
%pip install pyomo

  Using cached pyomo-6.9.5-cp313-cp313-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (9.1 kB)
  Using cached ply-3.11-py2.py3-none-any.whl.metadata (844 bytes)
Using cached pyomo-6.9.5-cp313-cp313-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl (4.4 MB)
Using cached ply-3.11-py2.py3-none-any.whl (49 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pyomo]32m1/2 [pyomo]
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pyomo.environ as pyo

battery capacity less than or equal to off peak requirement

stored power = peak_requirement - solar_capacity

off peak cost = off peak grid price *  (off peak requirement - solar capacity)

peak cost = peak requirement - solar capacity

minimize total cost

In [15]:
model = pyo.ConcreteModel()
model.solar_capacity = pyo.Var(within=pyo.NonNegativeReals)
# model.battery_capacity = pyo.Var(within=pyo.NonNegativeReals, initialize=0)
model.peak_price = pyo.Param(initialize=0.25)
model.peak_requirement = pyo.Param(initialize=10)
# model.off_peak_price = pyo.Param(initialize=0.25)
# model.off_peak_requirement = pyo.Param(initialize=5)


# model.PeakConstraint = pyo.Constraint(expr=model.grid + model.SolarCapacity == 900)
# model.total_power_constraint = pyo.Constraint(
#    expr=model.solar_capacity <= model.battery_capacity + model.peak_requirement
# )

model.peak_grid_usage = pyo.Var(within=pyo.NonNegativeReals)

model.minimizeCost = pyo.Objective(
    expr=model.peak_grid_usage,
    sense=pyo.minimize,
)

model.abs_constraint1 = pyo.Constraint(
    expr=model.peak_grid_usage >= model.peak_requirement - model.solar_capacity
)
model.abs_constraint2 = pyo.Constraint(
    expr=model.peak_grid_usage >= -(model.peak_requirement - model.solar_capacity)
)

optimizer = pyo.SolverFactory("glpk")
optimizer.solve(model)
print(model.display())

Model unknown

  Variables:
    solar_capacity : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  10.0 :  None : False : False : NonNegativeReals
    peak_grid_usage : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeReals

  Objectives:
    minimizeCost : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :   0.0

  Constraints:
    abs_constraint1 : Size=1
        Key  : Lower : Body : Upper
        None :  None :  0.0 :   0.0
    abs_constraint2 : Size=1
        Key  : Lower : Body : Upper
        None :  None : -0.0 :   0.0
None
